In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from glob2 import glob
import folium
from folium.plugins import FeatureGroupSubGroup
from folium.plugins import FastMarkerCluster
from folium.plugins import MiniMap
from folium.plugins import DualMap

### Load observations

In [2]:
#load data
data_folder = '../data/observations_final'
df = pd.concat([pd.read_csv(f).assign(challenge=f.replace('.csv','')) for f in glob(data_folder+'/CNC_San_Francisco_*.csv')])

In [3]:
#create year 
df['year']=df['time_observed_at'].str[0:4].astype('int64')

In [4]:
df.shape

(139040, 40)

In [5]:
df.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,year
0,5677489,Thu Apr 13 2017 12:40:50 GMT-0700 (PDT),2017-04-13T12:40:50-07:00,America/Los_Angeles,2017-04-13T13:29:41-07:00,2018-11-26T12:33:26-08:00,NaN,8778,gyrrlfalcon,needs_id,...,family,49025,False,False,False,127627,127627,"[{'user_id': 8778, 'category': 'maverick', 'di...",../data/observations_final\CNC_San_Francisco_2017,2017
1,5682279,Thu Apr 13 2017 15:57:53 GMT-0700 (PDT),2017-04-13T15:57:53-07:00,America/Los_Angeles,2017-04-13T18:37:08-07:00,2018-11-26T06:03:56-08:00,NaN,4797,paranger,research,...,species,48245,True,False,False,8756,8756,"[{'user_id': 179103, 'category': 'supporting',...",../data/observations_final\CNC_San_Francisco_2017,2017
2,5682371,Thu Apr 13 2017 16:08:07 GMT-0700 (PDT),2017-04-13T16:08:07-07:00,America/Los_Angeles,2017-04-13T18:44:41-07:00,2018-11-26T06:03:57-08:00,NaN,4797,paranger,research,...,species,48245,True,False,False,8756,8756,"[{'user_id': 179103, 'category': 'supporting',...",../data/observations_final\CNC_San_Francisco_2017,2017
3,5684642,2017/04/14 12:13 AM PDT,2017-04-14T00:13:00-07:00,America/Los_Angeles,2017-04-14T00:20:21-07:00,2017-04-21T14:58:31-07:00,NaN,1,kueda,needs_id,...,family,55518,False,False,False,6696,6696,"[{'user_id': 1, 'category': None, 'disagreemen...",../data/observations_final\CNC_San_Francisco_2017,2017
4,5684660,Fri Apr 14 2017 00:28:12 GMT-0700 (PDT),2017-04-14T00:28:12-07:00,America/Los_Angeles,2017-04-14T00:29:36-07:00,2020-02-15T14:31:49-08:00,NaN,3494,damontighe,needs_id,...,subgenus,67740,False,False,False,1246,1246,"[{'user_id': 3494, 'category': 'leading', 'dis...",../data/observations_final\CNC_San_Francisco_2017,2017


In [6]:
#check if there are missing latitudes and longitudes
df['latitude'].isna().sum(), df['longitude'].isna().sum()

(0, 0)

### Map

In [9]:
#base map
#coordinates for San Francisco
sf=[37.773972,-122.431297]
m = folium.Map(location=sf, zoom_start=12, tiles=None, control_scale=True)

#add tile and title 
folium.TileLayer('openstreetmap', name='San Francisco OSM').add_to(m)

#add observations
fg = folium.FeatureGroup(name='San Francisco (observations)')
m.add_child(fg)

Y2018 = FeatureGroupSubGroup(fg, '2018 observations', show=False)
m.add_child(Y2018)

Y2019 = FeatureGroupSubGroup(fg, '2019 observations', show=False)
m.add_child(Y2019)

Y2020 = FeatureGroupSubGroup(fg, '2020 observations')
m.add_child(Y2020)

df[df['year'].eq(2018)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                             .add_to(Y2018), axis=1)

df[df['year'].eq(2019)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                             .add_to(Y2019), axis=1)

df[df['year'].eq(2020)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                             .add_to(Y2020), axis=1)

#add clusters
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: 'map-marker', markerOpacity: 0, className: 'invisible-marker'});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

clusters=FastMarkerCluster([list(a) for a in zip(np.asarray(df.latitude), np.asarray(df.longitude))],
                           callback=callback,
                           show=False,
                           name='clusters (all years)')
clusters.add_to(m)

#clusters over years
c18=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2018)].latitude),
                                                 np.asarray(df[df['year'].eq(2018)].longitude))],
                           callback=callback,
                           show=False,
                           name='clusters (2018)')
c18.add_to(m)

c19=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2019)].latitude),
                                                 np.asarray(df[df['year'].eq(2019)].longitude))],
                           callback=callback,
                           show=False,
                           name='clusters (2019)')
c19.add_to(m)

c20=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2020)].latitude),
                                                 np.asarray(df[df['year'].eq(2020)].longitude))],
                           callback=callback,
                           name='clusters (2020)')
c20.add_to(m)

#add layer control
folium.LayerControl(collapsed=False).add_to(m)

#add minimap
minimap = MiniMap(width=90, height=90, toggle_display=True, zoom_animation=True)
m.add_child(minimap)

#save map
m.save('../maps/map_san_francisco_complete_final_osm.html')

#call map
#m

### Dual Map - before (2019) and during (2020) pandemic 

In [8]:
#base map
#coordinates for San Francisco
sf=[37.773972, -122.431297]
mdual_osm=DualMap(location=sf, tiles=None, zoom_start=11, control_scale=True, layout='vertical')

#add tile and title 
folium.TileLayer('openstreetmap', name='San Francisco 2020').add_to(mdual_osm.m2)
folium.TileLayer('openstreetmap', name='San Francisco 2019').add_to(mdual_osm.m1)


#add observations
fg_m2 = folium.FeatureGroup(name='San Francisco (2020)', control=False)
mdual_osm.m2.add_child(fg_m2)

fg_m1 = folium.FeatureGroup(name='San Francisco (2019)', control=False)
mdual_osm.m1.add_child(fg_m1)

Y2020 = FeatureGroupSubGroup(fg_m2, 'observations (2020)')
mdual_osm.m2.add_child(Y2020)

Y2019 = FeatureGroupSubGroup(fg_m1, 'observations (2019)')
mdual_osm.m1.add_child(Y2019)


df[df['year'].eq(2019)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                                   .add_to(Y2019), axis = 1)

df[df['year'].eq(2020)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                                   .add_to(Y2020), axis=1)

#add clusters
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: 'map-marker', markerOpacity: 0, className: 'invisible-marker'});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

c19_dual=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2019)].latitude),
                                                 np.asarray(df[df['year'].eq(2019)].longitude))],
                           callback=callback,
                           name='clusters (2019)')
c19_dual.add_to(mdual_osm.m1)

c20_dual=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2020)].latitude),
                                                 np.asarray(df[df['year'].eq(2020)].longitude))],
                           callback=callback,
                           name='clusters (2020)')
c20_dual.add_to(mdual_osm.m2)


#add layer control
folium.LayerControl(collapsed=False).add_to(mdual_osm)

minimap = MiniMap(width=90, height=90, toggle_display=True, zoom_animation=True)
mdual_osm.add_child(minimap)

#mdual_osm.save('../maps/map_san_francisco_osm_2019_2020_dualmap.html')

#mdual_osm